In [1]:
from google.colab import files
from zipfile import ZipFile

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!pip install BERTopic
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.8 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 662 kB 52.9 MB/s 
     |████████████████████████████████| 5.2 MB 20.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 5.8 MB 45.8 MB/s 
     |████████████████████████████████| 1.3 MB 58.4 MB/s 
     |████████████████████████████████| 182 kB 55.0 MB/s 
     |████████████████████████████████| 7.6 MB 58.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700857 sha256=fea708209ad031302d0fa9bcfff7237dda0fc2be7e34b15e5e1dd2197b333741
  Stored in directory: /root/.cache/

In [3]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

In [ ]:
files.upload()

In [12]:
zip = ZipFile("documentos.zip")
zip.extractall()

In [13]:
import glob
dados = []
dir = "/content/documentos/"
for arquivo in glob.iglob(dir + '*.txt', recursive=False):
  meuarquivo = open(arquivo, "r", encoding="utf8")
  content = meuarquivo.read()
  dados.append(content)
  meuarquivo.close()

In [14]:
len(dados)

22

In [ ]:
dados[3]

In [16]:
sentencas = []
for item in dados:
  for t in sent_tokenize(item):
    sentencas.append(t)

In [17]:
len(sentencas)

1504

In [18]:
sentencas[300]


'Imunologia é o ramo da biologia que estuda o sistema imunitário (ou imunológico) de todos os organismos.'

In [19]:
modelo = BERTopic(language="portuguese", verbose=True, embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens",
                  top_n_words=15, min_topic_size=10, nr_topics=20,
                  vectorizer_model=CountVectorizer(ngram_range=(1,3),
                   stop_words=stopwords.words("portuguese"), min_df=10)) 

In [20]:
%%time
topicos, probabilidades = modelo.fit_transform(sentencas)

Downloading:   0%|          | 0.00/795 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/722 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

2022-12-30 12:59:12,818 - BERTopic - Transformed documents to Embeddings
2022-12-30 12:59:28,194 - BERTopic - Reduced dimensionality
2022-12-30 12:59:28,280 - BERTopic - Clustered reduced embeddings
2022-12-30 12:59:40,359 - BERTopic - Reduced number of topics from 30 to 21


CPU times: user 5min 3s, sys: 12.5 s, total: 5min 15s
Wall time: 5min 11s


In [21]:
frequencia = modelo.get_topic_info()
frequencia.head(10)

,Topic,Count,Name
0,-1,448,-1_grande_além_vez_estudo
1,0,140,0_tempo_maior_cada_meio
2,1,95,1_sistema_podem_através_processo
3,2,75,2_engenharia_profissional_engenheiro_áreas
4,3,72,3_métodos_através_pesquisa_estudo
5,4,66,4_ver_química_vez_muitos
6,5,66,5_controle_engenharia_ver_processos
7,6,63,6_origem_tipos_através_10
8,7,61,7_grandes_engenheiro_conhecimento_técnicas
9,8,55,8_produção_conhecimento_profissional_área


In [22]:
modelo.get_topic(7)


[('grandes', 0.09176757911107522),
 ('engenheiro', 0.08939071022751373),
 ('conhecimento', 0.07243563132256228),
 ('técnicas', 0.05673341493847398),
 ('engenharia', 0.051756979390917605),
 ('forma', 0.05099292185879583),
 ('estruturas', 0.04977171764009211),
 ('então', 0.044992142174689324),
 ('ramo', 0.04341507280991398),
 ('ainda', 0.042482904657833075),
 ('formação', 0.03670703164443009),
 ('muitas', 0.036487875507575845),
 ('pode', 0.034381042395197584),
 ('desde', 0.03374410663101699),
 ('desta', 0.03374410663101699)]

In [23]:
#modelo.visualize_barchart(n_words=15,top_n_topics=20 )
modelo.visualize_barchart(n_words=10,topics=[0,3] )


In [24]:
modelo.visualize_hierarchy(top_n_topics=20)

In [25]:
modelo.visualize_heatmap(n_clusters=15)